# Firts  point

Solve the system (4.37) by using Multivariate Newton’s Method. Find the receiver
position (x, y, z) near earth and time correction d for known, simultaneous satellite
positions (15600, 7540, 20140), (18760, 2750, 18610), (17610, 14630, 13480),
(19170, 610, 18390) in km, and measured time intervals 0.07074, 0.07220,
0.07690, 0.07242 in seconds, respectively. Set the initial vector to be
(x 0 , y 0 , z 0 , d 0 ) = (0, 0, 6370, 0). As a check, the answers are approximately
(x, y, z) = (−41.77271, −16.78919, 6370.0596), and d = −3.201566 × 10 −3 seconds.

In [2]:
import  numpy as np
import scipy.linalg as la

# Error function  construction

def FR(x1,C):
    c = 299792.458  # speed of light in km/s
    x=x1[0]
    y=x1[1]
    z=x1[2]
    d=x1[3]
    R=(x-C[:,0])**2+ (y-C[:,1])**2+(z-C[:,2])**2-c**2*(C[:,3]-d)**2
    return R

# Jacobian error function  construction 
def DFR(x1,C):
    c = 299792.458  # speed of light in km/s
    x=x1[0]
    y=x1[1]
    z=x1[2]
    d=x1[3]
    N=len(C[:,1])

    s1=np.zeros([N,4])
    s1[:,0]=2.0*(x-C[:,0])
    s1[:,1]=2.0*(y-C[:,1])
    s1[:,2]=2.0*(z-C[:,2])
    s1[:,3]=2*c**2*(C[:,3]-d)
    return s1

def newton_gauss(FR,DFR,tol,N,x0,C):
    for i in range(N):
        A=DFR(x0,C)
        F1=FR(x0,C)
        F1=A.T@FR(x0,C)
        A=A.T@A
        if abs(la.det(A))  < 10**-9:
            print('El sistema no se puede resolver, porque A es singular,intente otro punto inicial')
            break
        x=la.solve(A,-F1)
        if np.linalg.norm(x) < tol:
            print('el sistema se soluciono en la iteracion:',i)
            break
        x0=x0+x
        
        return x




In [4]:
C=np.zeros([4,4])
C[0,:]=np.array([15600,7540,20140,0.07074])
C[1,:]=np.array([18760,2750,18610,0.07220])
C[2,:]=np.array([17610,14630,13480,0.07690])
C[3,:]=np.array([19170,610,18390,0.07242])


# the varible C is a matrix
# C[:,0] first column of matrix has  information of 1 cordiante of each satellite
# C[:,1] Second column of matrix has  information of 2 cordiante of each satellite
# C[:,2] third column of matrix has  information of 3 cordiante of each satellite
# C[:,3] fourth column of matrix has  information of time od each satillite


tol=1e-1
N=2000
#inital point
x0=np.array([0, 0, 6370, 0])
x=newton_gauss(FR,DFR,1e-1,100,x0,C)
print(x)

[-4.17733669e+01 -1.67510571e+01  5.18313453e+00 -3.26271451e-03]


In [5]:
C=np.zeros([4,4])
C[0,:]=np.array([7194.3,-786.786,25565,0.0685])
C[1,:]=np.array([-6076.6,9078.5,24220,	0.0699])
C[2,:]=np.array([12587	,10119,	21098,	0.073])
C[3,:]=np.array([2834.4,	-2673.3,	26283,	0.0678])
x0=np.array([0, 0, 6370, 1.0e-4])
x=newton_gauss(FR,DFR,1e-5,100,x0,C)
print(x)

[7.77053938e+01 1.63153811e+02 4.00873497e+02 1.29433933e-03]


# Second point



# Fourt point

Now set up a test of the conditioning of the GPS problem. Define satellite positions
$(A_i , B_i , C_i )$ from spherical coordinates $(\rho,\phi_i ,\theta_i )$ as
$$A_i = \rho\cos \phi_i  \cos \theta_i$$
$$B_i = \rho\cos \phi_i \sin \theta_i$$
$$C_i = \rho \sin \phi_i,$$

where $\rho = 26570$ km is fixed, while $0\leq \phi_1\leq\pi/2$  and $0 \leq \theta_i \theta\leq 2*\pi$ for $i = 1,\dots, 4$ are
chosen arbitrarily. The $\phi$ coordinate is restricted so that the four satellites are in the
upper hemisphere. Set $x = 0$, $y = 0$, $z = 6370$, $d = 0.0001$, and calculate the
corresponding satellite ranges $R_i = \sqrt{A_i^2 + B_i^2 + (C_i − 6370)^2}$ and travel times
$t_i = d + R_i /c$.

In [6]:
from numpy.random import default_rng
def coord_spherical(X=np.array([0,0,6370,-3.201566e-3]), n_satelites=4):
    # n_satelites = number of satelites
    # X = vector of random numbers
    #x[0]=position in x
    #x[1]=position in y
    #x[2]=position in z
    #x[3]=time correction
    rho = 26570
    rng = np.random.default_rng(seed=42) # semente  
    c = 299792.458 #velocidad de la luz
    
    Ai = np.zeros([n_satelites,1])
    Bi = np.zeros([n_satelites,1])
    Ci = np.zeros([n_satelites,1])
    ti = np.zeros([n_satelites,1])
    for i in range(n_satelites):
        phi_max = np.pi/2
        phi = (phi_max-0)*rng.random((1, 1)) + 0

        theta_max = 2*np.pi
        theta = (theta_max-0)*rng.random((1, 1)) + 0

        Ai[i] = rho*np.cos(phi)*np.cos(theta)
        Bi[i] = rho*np.cos(phi)*np.sin(theta)
        Ci[i] = rho*np.sin(phi)
        R = np.sqrt((Ai[i]-X[0])**2 + (Bi[i]-X[1])**2 + (Ci[i]-X[3])**2)
      
        ti[i] = X[3] + R / c
   
    
    Matrix=np.zeros([n_satelites,4])
    Matrix[:,0]=Ai.T
    Matrix[:,1]=Bi.T
    Matrix[:,2]=Ci.T
    Matrix[:,3]=ti.T
    
    # each line of the matrix has information of each satelite (x,y,z,time)
    return Matrix


In [9]:
def point_4():
    global_max = 0
    max_error_pos = 0
    tol=1e-15
    N_max_ite=200
    c = 299792.458 #speed of light in km/s
    n_satelites=4
    total_cases=2**n_satelites
    X=[0,0,6370,0.0001]
    sphe = coord_spherical(X,n_satelites)
    X1=newton_gauss(FR,DFR,tol, N_max_ite,x0,sphe)
    for cases in range(total_cases):
        
        t=sphe[:,3]
        
        signo=(2 * np.random.randint (0,2, size = (n_satelites)) - 1)
        dt=1e-8*signo
        new_t = t + dt
        sphe[:,3]=new_t
        x=newton_gauss(FR,DFR,tol, N_max_ite,x0,sphe)
       


        #factor de magnificacion del error
        f_error = np.linalg.norm(x[1:3] - X1[1:3],np.inf)
        b_error = c*1e-8
        emf = f_error/b_error
        if emf > global_max:
            global_max = emf #se guarda el valor maximo de emf
            max_error_pos = f_error #se guarda el valor maximo de error de posicion
    return global_max, max_error_pos

global_max, max_error_pos = point_4()
max_error_pos_4 = max_error_pos*1000
print(max_error_pos_4)

413.6531166459463


879.644302323868
